In [1]:
import cv2
import numpy as np;
import matplotlib.pyplot as plt;
from mpl_toolkits import mplot3d
from scipy.spatial import KDTree
import scipy
%matplotlib inline

import trimesh
import pyglet
from scipy.spatial import Delaunay

# Trying to get Meshes using Data

## Fowlke's Code Modified by me

In [2]:
from camutils import Camera

In [3]:
def residuals(pts3,pts2,K,params):
    """ 
    """

    r1,r2,r3,t1,t2,t3 = params
    R, _ = cv2.Rodrigues( np.array([[r1,r2,r3]]) )
    t = np.array([[t1,t2,t3]]).T
    pts2_proj = K @ ((R.T @ pts3) - t)
    pts2_proj = (pts2_proj / pts2_proj[2,:])[0:2,:]
    residual = pts2 - pts2_proj

    return residual.flatten()

def calibratePose(pts3,pts2,K,R0,t0):
    """
    """
    rvec,_ = cv2.Rodrigues(R0)
    r1,r2,r3 = rvec[:,0]
    t1,t2,t3 = t0[:,0]
    
    params_init = [r1,r2,r3,t1,t2,t3]
    # define our error function
    efun = lambda params: residuals(pts3,pts2,K,params)
    popt,_ = scipy.optimize.leastsq(efun,params_init)
    r1,r2,r3,t1,t2,t3 = popt
    R,_ = cv2.Rodrigues(np.array([[r1,r2,r3]]))
    t = np.array([[t1,t2,t3]]).T
    return R,t


### My Aruco Code for pose estimation

In [5]:
class arucoTest:
    def __init__(self, frame, intr, dist, depth, markerLength=0.0246153846):
        self.arucoPoseDict = {}

        self.arucoDict = cv2.aruco.Dictionary_get( cv2.aruco.DICT_6X6_50)
        self.arucoParams = cv2.aruco.DetectorParameters_create()

        self.markerLength = markerLength

        corners, idx, _ = cv2.aruco.detectMarkers(frame, self.arucoDict, parameters=self.arucoParams)
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, self.markerLength, intr, dist)

        self.K = intr
        self.D = dist
        
        self.corners0 = corners
        self.idx0 = idx
        self.frame0 = frame
        self.depth0 = depth
        for i in range( idx.shape[0] ):
            index = idx[i,0]
            R,_ = cv2.Rodrigues(rvecs[i])
            t = tvecs[i].T
            self.arucoPoseDict[index] = (R,t)
    def getPose(self, frame):
        corners, idx, _ = cv2.aruco.detectMarkers(frame, self.arucoDict, parameters=self.arucoParams)
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, self.markerLength, self.K, self.D)


        avgR = np.zeros((3,3))
        avgT = np.zeros((3,1))
        count = 0

        for i in range(idx.shape[0]):
            index = idx[i,0]
            R1,_ = cv2.Rodrigues(rvecs[i])
            t1 = tvecs[i].T

            if index in self.arucoPoseDict.keys():
                R,t = self.arucoPoseDict[index]
                poseR = R @ R1.T
                poseT = t - (R @ R1.T) @ t1

                avgR += poseR
                avgT += poseT
                count += 1
        if count != 0:
            avgR /= count
            avgT /= count
        return avgR, avgT
    def getClouds(self, frame, depth):
        corners, idx, _ = cv2.aruco.detectMarkers(frame, self.arucoDict, parameters=self.arucoParams)
        

        indexes, corrFrame, corrFrame0 = np.intersect1d( idx[:,0], self.idx0[:,0] , return_indices= True)
        #assumes there are matches
        pts2 = corners[ corrFrame[0] ][0].T
        pts2_0 = self.corners0[ corrFrame0[0] ][0].T
        
        for i in range(indexes.shape[0]):
            #corners[i] is of shape (1,4,2)
            pts2 = np.hstack((pts2, corners[ corrFrame[i] ][0].T))
            pts2_0 = np.hstack((pts2_0, self.corners0[ corrFrame0[i] ][0].T))
        
        pts3_0 = self.triangulate(pts2_0,self.depth0)
        
        return pts2, pts3_0
    def triangulate(self, pts2, depth):
        indexes = np.uint8(pts2)
        pts3 = np.zeros((3,pts2.shape[1]),dtype=np.float32)
        for i in range(indexes.shape[1]):
            y,x = indexes[:,i]
            z = depth[y,x]
            x3 = (x - K[0,2]) * z / K[0,0]
            y3 = (y - K[1,2]) * z / K[1,1]
            pts3[:,i] = np.array([x3,y3,z])
        return pts3
    def updateEstimate(self, pts2, pts3, R,t):
        cam1 = Camera(K[0,0], c=np.array([[K[0,2],K[1,2]]]), R=R,t=t )
        Rf,tf = calibratePose(pts3,pts2,self.K,R,t)
        return Rf,tf

## My triangulation code

In [6]:
def triangulate(depth_map, fx = 5, fy = 5):
    h,w = depth_map.shape
    xx,yy = np.meshgrid(np.arange(w), np.arange(h))
    mask = (depth_map > 0)

    proj_x = (xx[mask] - w/2) * depth_map[mask] / fx
    proj_y = (yy[mask] - h/2) * depth_map[mask] / fy

    return np.vstack((proj_x.flatten(), proj_y.flatten(), depth_map[mask].flatten())), mask

## Stuff

In [7]:
def create_output(vertices, colors, filename):
	vertices = np.hstack([vertices,colors])

	ply_header = '''ply
		format ascii 1.0
		element vertex %(vert_num)d
		property float x
		property float y
		property float z
		property uchar red
		property uchar green
		property uchar blue
		end_header
		'''
	with open(filename, 'w') as f:
		f.write(ply_header %dict(vert_num=len(vertices)))
		np.savetxt(f,vertices,'%f %f %f %d %d %d')

## Getting to the code

In [8]:
with open('testdata.npy', 'rb') as f:
    im0 = np.load(f)
    d0 = np.load(f)
    
    im1= np.load(f)
    d1= np.load(f)
    
    im2= np.load(f)
    d2= np.load(f)
    
    im3= np.load(f)
    d3= np.load(f)
    
    im4= np.load(f)
    d4= np.load(f)
    
    im5= np.load(f)
    d5= np.load(f)
    
with open('../calibration/intrinsics.npy', 'rb') as f:
    K = np.load(f)
    dist = np.load(f)

In [9]:
pts3_0,mask0 = triangulate(d0, K[0,0], K[1,1])
pts3_1,mask1 = triangulate(d1, K[0,0], K[1,1])
pts3_2,mask2 = triangulate(d2, K[0,0], K[1,1])

aruco = arucoTest(im0, K, dist, d0, 0.04445 / 0.0002500000118743628)
R1,t1 = aruco.getPose(im1)
R2,t2 = aruco.getPose(im2)

pts3_1t = R1 @ pts3_1 + t1
pts3_2t = R2 @ pts3_2 + t2

In [10]:
colormap0 = im0[:,:,::-1][mask0].reshape(-1,3)
colormap1 = im1[:,:,::-1][mask1].reshape(-1,3)
colormap2 = im2[:,:,::-1][mask2].reshape(-1,3)

In [11]:
pts3s = np.hstack((pts3_0,pts3_1t))
colormaps = np.vstack((colormap0,colormap1))
create_output(pts3s.T,colormaps,'beforeAlign.ply')

In [12]:
pts2, pts3 = aruco.getClouds(im1,d1)

In [13]:
R1_,t1_ = aruco.updateEstimate(pts2,pts3,R1,t1)

In [15]:
pts3_1t = R1_ @ pts3_1 + t1_

In [16]:
pts3s = np.hstack((pts3_0,pts3_1t))
colormaps = np.vstack((colormap0,colormap1))
create_output(pts3s.T,colormaps,'afterAlign.ply')